## initialization (2025-12-19-09-14)

In [5]:
# ============================================================================
# IMPORTS
# ============================================================================
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# Build the RAG chain (reusable for multiple questions)
from langchain_core.runnables import RunnablePassthrough

In [3]:
# ============================================================================
# ENV SETUP (fail-fast)
# ============================================================================
load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL")  # e.g. https://openrouter.ai/api/v1
if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY in .env")
if not OPENROUTER_BASE_URL:
    raise RuntimeError("Missing OPENROUTER_BASE_URL in .env")

In [4]:
# Initialize embedding model for converting text into numerical vectors
# Used during: document indexing and query embedding for retrieval
embeddings = OpenAIEmbeddings(
    api_key=OPENROUTER_API_KEY,        # API key for authentication
    base_url=OPENROUTER_BASE_URL,      # OpenRouter endpoint URL
    model="text-embedding-3-small",    # Small, fast embedding model (1536 dimensions)
)

# Initialize the LLM for generating natural language responses
# Used during: answer generation phase of RAG
llm = ChatOpenAI(
    api_key=OPENROUTER_API_KEY,        # API key for authentication
    base_url=OPENROUTER_BASE_URL,      # OpenRouter endpoint URL
    model="openai/gpt-4o",             # GPT-4o model for high-quality responses
    temperature=0.2,                   # Low temperature for more deterministic, focused answers
)

## Indexing pipeline for RAG (2025-12-19-09-14)

"""
PyCharm (With LangChain) — Simple RAG from plain text (data.txt)
- LLM: OpenRouter (ChatOpenAI)
- Retriever: FAISS (in-memory)
"""

In [7]:
# ============================================================================
# DOCUMENT INDEXING PIPELINE
# ============================================================================
# 1. LOAD: Read source documents from disk
# 2. SPLIT: Chunk documents into smaller pieces for embedding
# 3. EMBED: Convert chunks to vectors using embedding model
# 4. STORE: Save vector index to disk (FAISS)
# 5. RETRIEVE: Initialize retriever for similarity search
# ============================================================================

DB_FAISS_PATH = "../Assets/Data"  # Directory to save the index
INDEX_NAME = "index"  # Default index name used by FAISS

# Check if the vector store files actually exist
index_file = os.path.join(DB_FAISS_PATH, f"{INDEX_NAME}.faiss")
pkl_file = os.path.join(DB_FAISS_PATH, f"{INDEX_NAME}.pkl")

if os.path.exists(index_file) and os.path.exists(pkl_file):
    print(f"Loading existing vector store from: {DB_FAISS_PATH}")

    vs = FAISS.load_local(
        folder_path=DB_FAISS_PATH,
        embeddings=embeddings,
        allow_dangerous_deserialization=True
    )
else:
    print("Index not found. Creating new vector store...")

    # 1. Load
    docs = TextLoader("data.txt", encoding="utf-8").load()

    # 2. Split
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=80,
    )
    splits = splitter.split_documents(docs)

    # 3. Embed & Create Vector Store
    vs = FAISS.from_documents(splits, embeddings)

    # 4. Create directory if it doesn't exist
    os.makedirs(DB_FAISS_PATH, exist_ok=True)

    # 5. Save to disk
    vs.save_local(DB_FAISS_PATH)
    print(f"Vector store saved to: {DB_FAISS_PATH}")

# Initialize Retriever
retriever = vs.as_retriever(search_kwargs={"k": 3})

Loading existing vector store from: ../Assets/Data


### print/debug the variables to see their values (optional)

In [8]:
# Print the variables to inspect their values
print("DB_FAISS_PATH:", DB_FAISS_PATH)
print("INDEX_NAME:", INDEX_NAME)
print("index_file:", index_file)
print("pkl_file:", pkl_file)
print("index_file exists:", os.path.exists(index_file))
print("pkl_file exists:", os.path.exists(pkl_file))

DB_FAISS_PATH: ../Assets/Data
INDEX_NAME: index
index_file: ../Assets/Data\index.faiss
pkl_file: ../Assets/Data\index.pkl
index_file exists: True
pkl_file exists: True


In [ ]:

    print("Index not found. Creating new vector store...")

    # 1. Load
    docs = TextLoader("data.txt", encoding="utf-8").load()

    # Print docs as list
    print("\n=== Documents as List ===")
    print(f"docs = {docs}")

    # 2. Split
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=80,
    )
    splits = splitter.split_documents(docs)

In [23]:
# Print the loaded documents
print("\n=== Original Documents ===")
print(f"Number of documents loaded: {len(docs)}")
for i, doc in enumerate(docs):
    print(f"\nDocument {i + 1}:")
    print(f"Content length: {len(doc.page_content)} characters")
    print(f"First 200 chars: {doc.page_content[:200]}...")
    print(f"Metadata: {doc.metadata}")

# Print the split documents
print("\n=== Split Documents ===")
print(f"Number of chunks after splitting: {len(splits)}")
for i, split in enumerate(splits[:5]):  # Show first 5 chunks
    print(f"\nChunk {i + 1}:")
    print(f"Content length: {len(split.page_content)} characters")
    print(f"Content: {split.page_content}")
    print(f"Metadata: {split.metadata}")

if len(splits) > 5:
    print(f"\n... and {len(splits) - 5} more chunks")


=== Original Documents ===
Number of documents loaded: 1

Document 1:
Content length: 614 characters
First 200 chars: ชื่อบริษัท : ABC  

ข้อมูลธุรกิจ : จัดจำหน่ายของเล่นและหนังสือการ์ตูน

ผู้ก่อตั้ง : ก้องรักสยาม

ปีที่ก่อตั้ง : 2568

สินค้าและบริการ
- ของเล่น: ฟิกเกอร์ โมเดล ตุ๊กตา บอร์ดเกม การ์ดเกม ของสะสม
- หนังส...
Metadata: {'source': 'data.txt'}

=== Split Documents ===
Number of chunks after splitting: 2

Chunk 1:
Content length: 480 characters
Content: ชื่อบริษัท : ABC  

ข้อมูลธุรกิจ : จัดจำหน่ายของเล่นและหนังสือการ์ตูน

ผู้ก่อตั้ง : ก้องรักสยาม

ปีที่ก่อตั้ง : 2568

สินค้าและบริการ
- ของเล่น: ฟิกเกอร์ โมเดล ตุ๊กตา บอร์ดเกม การ์ดเกม ของสะสม
- หนังสือการ์ตูน: มังงะ คอมิกส์ ไลท์โนเวล นิยายภาพ
- บริการเสริม: พรีออเดอร์สินค้าหายาก จัดกิจกรรมแฟนมีตติ้ง งานสะสม

กลุ่มเป้าหมาย
- เด็กและวัยรุ่นที่ชื่นชอบการ์ตูนและของเล่น
- นักสะสมฟิกเกอร์และโมเดล

ช่องทางการจัดจำหน่าย
- ร้านก้องรักสยาม
- ออนไลน์: เว็บไซต์, Facebook, Shopee, Lazada
Metadata: {'source': 'data.txt'}

Chunk 2:
Content l

## Generation pipeline for RAG (2025-12-19-09-14)

In [9]:
# ============================================================================
# RAG CHAIN
# ============================================================================
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Answer using ONLY the provided context. "
     "If the answer is not in the context, say you don't know."),
    ("human",
     "Question: {question}\n\nContext:\n{context}")
])

def format_docs(docs_):
    return "\n\n".join(d.page_content for d in docs_)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# no main() (your default)
question = "บริษัท ABC ขายอะไรบ้าง และมีช่องทางขายอะไร?"

In [10]:
answer = chain.invoke(question)

In [11]:
# For debugging/display: retrieve context separately
context_docs = retriever.invoke(question)

print("Q:", question)
print("\n--- Retrieved Context ---")
print(format_docs(context_docs))
print("\n--- Answer ---")
print(answer)

Q: บริษัท ABC ขายอะไรบ้าง และมีช่องทางขายอะไร?

--- Retrieved Context ---
ชื่อบริษัท : ABC  

ข้อมูลธุรกิจ : จัดจำหน่ายของเล่นและหนังสือการ์ตูน

ผู้ก่อตั้ง : ก้องรักสยาม

ปีที่ก่อตั้ง : 2568

สินค้าและบริการ
- ของเล่น: ฟิกเกอร์ โมเดล ตุ๊กตา บอร์ดเกม การ์ดเกม ของสะสม
- หนังสือการ์ตูน: มังงะ คอมิกส์ ไลท์โนเวล นิยายภาพ
- บริการเสริม: พรีออเดอร์สินค้าหายาก จัดกิจกรรมแฟนมีตติ้ง งานสะสม

กลุ่มเป้าหมาย
- เด็กและวัยรุ่นที่ชื่นชอบการ์ตูนและของเล่น
- นักสะสมฟิกเกอร์และโมเดล

ช่องทางการจัดจำหน่าย
- ร้านก้องรักสยาม
- ออนไลน์: เว็บไซต์, Facebook, Shopee, Lazada

ข้อมูลติดต่อ
- สำนักงานใหญ่: หมู่บ้านโคโนฮะ 55555
- โทรศัพท์: 089-123-4567
- อีเมล: info@eiei555.co.th
- เว็บไซต์: www.eiei555.co.th

--- Answer ---
บริษัท ABC ขายของเล่นและหนังสือการ์ตูน โดยมีช่องทางการจัดจำหน่ายผ่านร้านก้องรักสยาม และออนไลน์ผ่านเว็บไซต์, Facebook, Shopee, และ Lazada.
